<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/basicSuperRestoration/tests/Real_ESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-ESRGAN Super Restoration Upscaler

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2107.10833)
[![GitHub Stars](https://img.shields.io/github/stars/xinntao/Real-ESRGAN?style=social)](https://github.com/xinntao/Real-ESRGAN)


# 1. Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

In [1]:
!nvidia-smi

Wed Mar 30 10:51:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

1. linking next cloud

In [ ]:
# we'll link the dataset from next-cloud
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py

from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

input_folder, output_folder = create_io(database=nextcloud,topic='basicSuperRestoration',library='ESRGAN')

2. Clone Git repository

In [ ]:
import os
root_path = '/content/BasicSR'

# clone the repository
if not os.path.exists('BasicSR'):
  !git clone https://github.com/xinntao/Real-ESRGAN {root_path}

%ls

3. Set up the environment

In [ ]:
%cd {root_path}

# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

Only needed when we have to "**restart the Runtime**", otherwise 
the variables aren't defined.

In [ ]:
# populate vars when runtime restart is needed
input_folder = '/content/database/basicSuperRestoration/input'
output_folder = '/content/database/basicSuperRestoration/ESRGAN'
root_path = '/content/BasicSR'

print ('input ::: ' , input_folder)
print ('output ::: ' , output_folder)
print ('root ::: ' , root_path)

4. Download the pre-trained model

In [ ]:
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P /content/BasicSR/experiments/pretrained_models

# 2. Inference


In [ ]:
# vars
print (input_folder)
print (output_folder)

# change dir
%cd {root_path}

# Arguments
# -n, --model_name: Model names
# -i, --input: input folder or image
# -o, --output: output folder or image
# -s, --outscale: Output scale, can be arbitrary scale factor (ex. 3.5).
# -t, --tile: if out of memory, try to use this option. Tile size, 0 for no tile during testing

!python inference_realesrgan.py -n RealESRGAN_x4plus -i {input_folder} -o {output_folder} --outscale 3.5 --half --face_enhance

# 3. Visualization

In [ ]:
# utils for visualization
import os
import glob
import cv2
import matplotlib.pyplot as plt

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('Real-ESRGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# making a list of both I/O folders
input_list = sorted(glob.glob(os.path.join(input_folder, '*.jpg')))
output_list = sorted(glob.glob(os.path.join(output_folder, '*.jpg')))

# display each image in the upload folder
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)

# 4. Download Results


In [15]:
# Download the results
zip_filename = 'Real-ESRGAN_result.zip'

download_zip(zip_filename, output_folder)

NameError: ignored